# Imports

In [ ]:
import import_ipynb
from AlexFunctions import plot_images

import cv2
import numpy as np

# Visualize Results

In [ ]:
def compute_results(mission_numbers, bboxs, heatmaps, heatmap_colors, roof_masks, percent_roof_pixels, debug_images):
    
    # ----------------------------------------
    
    # superimpose all bounding boxes onto mission
    # TODO
    superimposed_bbox_masks = []
#     for i in range(0, len(bboxs)):
    
    
    if debug_images:
        plot_images(superimposed_bbox_masks, ['superimposed_bbox_masks ' + str(mission_number) for mission_number in mission_numbers])
    
    # ----------------------------------------
    
    # combine heatmap and roof mask into single image
    heatmap_roof_combinations = []
    for i in range(0, len(mission_numbers)):
        heatmap_roof_combinations.append(cv2.bitwise_or(heatmaps[i], roof_masks[i].astype(int)))
    
    if debug_images:
        plot_images(heatmap_roof_combinations, ['heatmap_roof_combinations ' + str(mission_number) for mission_number in mission_numbers])
    
    # ----------------------------------------
    
    # table of metadata
    metadata = []
    for i in range(0, len(mission_numbers)):
        data = dict()
#         data['num_anomalies'] =  # TODO - from superimposed_bbox_masks
        data['percent_roof_anomalies'] = percent_roof_pixels[i]
        
        heatmap_data = []
        num_non_black_pixels = np.sum(heatmaps[i] != [0, 0, 0])
        for c in heatmap_colors:
            heatmap_data.append(np.sum(heatmaps[i] == c) / num_non_black_pixels * 100)
        
        data['heatmap_histogram'] = heatmap_data
        
        metadata.append(data)
    
    print(metadata)
    
    # ----------------------------------------
    
    return superimposed_bbox_masks, heatmap_roof_combinations, metadata

In [ ]:
def display_results(mission_numbers, superimposed_bbox_masks, histogram_mission_images_bgr, heatmap_roof_combinations, metadata):
    # TODO
    # display superimposed bounding boxes on histogram_mission_images_bgr
    # display combined heatmap and roof mask
    # scatter plot of percent roof pixels & number of anomalies vs mission number
    # histogram of heatmap
    print('display_results')

In [ ]:
def run_visualize_results(mission_numbers, bboxs, histogram_mission_images_bgr, heatmaps, roof_masks, percent_roof_pixels, debug_images=False):
    superimposed_bbox_masks, heatmap_roof_combinations, metadata = compute_results(mission_numbers, bboxs, heatmaps, heatmap_colors, roof_masks, percent_roof_pixels, debug_images)
    display_results(mission_numbers, superimposed_bbox_masks, histogram_mission_images_bgr, heatmap_roof_combinations, metadata)

# Usage

In [ ]:
# run_visualize_results(mission_numbers, histogram_mission_images_bgr, bboxs, heatmaps, roof_masks, percent_roof_pixels, debug_images)